This notebook compiles scripts for determining coverage depth at genome positions where SNPs or indels have been introduced.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


Bad key savefig.frameon in file /Users/yeemayseah/anaconda3/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 421 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.level in file /Users/yeemayseah/anaconda3/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 472 ('verbose.level  : silent      # one of silent, helpful, debug, debug-annoying')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.2/matplotlibrc.template
or from the matplotlib source distribution

Bad key verbose.fileo in file /Users/yeemayseah/anaconda3/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 473 ('verbose.fileo  : sys.stdout  # a log filename, sys.stdout or sys.stderr')
You probably 

#### Get `.bed` files with MAPQ data

To get MAPQ at each position with ```bedtools```:

1. Create tab-separated genome file (mtb_genome.txt):

> NC_000962.3        4411532

2. Create window file (```-w``` for window size, ```-s``` for step size):

```singularity exec -B /molmicro,/molmicro/working/ymseah:/mnt /molmicro/common/singularity/bedtools-2.27.1-singularity-3.5.1.sif bedtools makewindows -g /mnt/read_mapping/mtb_genome.txt -w 1 -s 1 > /mnt/read_mapping/bedtools/mtb_genome.windows1.bed```

3. Create ```.bed``` file from ```.bam```:

```singularity exec -B /molmicro,/molmicro/working/ymseah:/mnt /molmicro/common/singularity/bedtools-2.27.1-singularity-3.5.1.sif bedtools bamtobed -i /mnt/deduped/picard/TEST-MTBREF_deduped_matecig.bam > read_mapping/bedtools/TEST-MTBREF_deduped_matecig.bed```

4. Map scores from ```.bed``` to window file. Default reports sum of scores:

```singularity exec -B /molmicro,/molmicro/working/ymseah:/mnt /molmicro/common/singularity/bedtools-2.27.1-singularity-3.5.1.sif bedtools map -a /mnt/read_mapping/bedtools/mtb_genome.windows1.bed -b /mnt/read_mapping/bedtools/TEST-MTBREF_deduped_matecig.bed > read_mapping/bedtools/TEST-MTBREF.mapq.windows1.bed```

4a. Use with ```-o collapse``` to report all scores per site:

```singularity exec -B /molmicro,/molmicro/working/ymseah:/mnt /molmicro/common/singularity/bedtools-2.27.1-singularity-3.5.1.sif bedtools map -a /mnt/read_mapping/bedtools/mtb_genome.windows1.bed -b /mnt/read_mapping/bedtools/TEST-MTBREF_deduped_matecig.bed -o collapse > read_mapping/bedtools/TEST-MTBREF.mapq_collapse.windows1.bed```

OR

4b. Use with ```-o distinct``` to report only unique scores per site:

```singularity exec -B /molmicro,/molmicro/working/ymseah:/mnt /molmicro/common/singularity/bedtools-2.27.1-singularity-3.5.1.sif bedtools map -a /mnt/read_mapping/bedtools/mtb_genome.windows1.bed -b /mnt/read_mapping/bedtools/TEST-MTBREF_deduped_matecig.bed -o distinct > read_mapping/bedtools/TEST-MTBREF.mapq_distinct.windows1.bed```

OR

4c. Use with ```-c 5,5,5,5``` (which columns in ```-b``` to apply following calculations) and ```-o min,max,median,count``` to get summary stats:

```singularity exec -B /molmicro,/molmicro/working/ymseah:/mnt /molmicro/common/singularity/bedtools-2.27.1-singularity-3.5.1.sif bedtools map -a /mnt/read_mapping/bedtools/mtb_genome.windows1.bed -b /mnt/read_mapping/bedtools/TEST-MTBREF_deduped_matecig_mq10.bed -c 5,5,5,5 -o min,max,median,count > read_mapping/bedtools/TEST-MTBREF_mq10.mapq_stats.windows1.bed```


In [3]:
mtbref = pd.read_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-MTBREF_mq10.mapq_stats.windows1.bed', sep='\t', header=None)
mtb20 = pd.read_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-H37Rv20snps_mq10.mapq_stats.windows1.bed', sep='\t', header=None)
mtb200 = pd.read_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-H37Rv200snps_mq10.mapq_stats.windows1.bed', sep='\t', header=None)
mtb2000 = pd.read_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-H37Rv2000snps_mq10.mapq_stats.windows1.bed', sep='\t', header=None)
mtb20003 = pd.read_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-H37Rv20003snps_mq10.mapq_stats.windows1.bed', sep='\t', header=None)
mtbvar1 = pd.read_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-MTBVAR1_mq10.mapq_stats.windows1.bed', sep='\t', header=None)
mtbvar2 = pd.read_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-MTBVAR2_mq10.mapq_stats.windows1.bed', sep='\t', header=None)

/Users/yeemayseah/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
mtbref.rename(columns={0:'chr', 1:'start', 2:'end', 3:'min_mapq', 4:'max_mapq', 5:'median_mapq', 6:'count_mapq'}, inplace=True)
mtb20.rename(columns={0:'chr', 1:'start', 2:'end', 3:'min_mapq', 4:'max_mapq', 5:'median_mapq', 6:'count_mapq'}, inplace=True)
mtb200.rename(columns={0:'chr', 1:'start', 2:'end', 3:'min_mapq', 4:'max_mapq', 5:'median_mapq', 6:'count_mapq'}, inplace=True)
mtb2000.rename(columns={0:'chr', 1:'start', 2:'end', 3:'min_mapq', 4:'max_mapq', 5:'median_mapq', 6:'count_mapq'}, inplace=True)
mtb20003.rename(columns={0:'chr', 1:'start', 2:'end', 3:'min_mapq', 4:'max_mapq', 5:'median_mapq', 6:'count_mapq'}, inplace=True)
mtbvar1.rename(columns={0:'chr', 1:'start', 2:'end', 3:'min_mapq', 4:'max_mapq', 5:'median_mapq', 6:'count_mapq'}, inplace=True)
mtbvar2.rename(columns={0:'chr', 1:'start', 2:'end', 3:'min_mapq', 4:'max_mapq', 5:'median_mapq', 6:'count_mapq'}, inplace=True)

In [14]:
mtb20

,chr,start,end,min_mapq,max_mapq,median_mapq,count_mapq
0,NC_000962.3,0,1,60,60,60,1
1,NC_000962.3,1,2,60,60,60,1
2,NC_000962.3,2,3,60,60,60,1
3,NC_000962.3,3,4,60,60,60,1
4,NC_000962.3,4,5,60,60,60,1
...,...,...,...,...,...,...,...
4411527,NC_000962.3,4411527,4411528,60,60,60,1
4411528,NC_000962.3,4411528,4411529,60,60,60,1
4411529,NC_000962.3,4411529,4411530,.,.,.,0
4411530,NC_000962.3,4411530,4411531,.,.,.,0


#### Get per-base genome coverage data

`singularity exec -B /molmicro,/molmicro/working/ymseah:/mnt /molmicro/common/singularity/bedtools-2.27.1-singularity-3.5.1.sif bedtools genomecov -ibam /mnt/deduped/picard/samtools_filtered/TEST-MTBREF_deduped_matecig_mq10.bam -d > read_mapping/bedtools/TEST-MTBREF_deduped_matecig_mq10_perbase_genomecov.txt`

In [2]:
mtbref_perbase_cov = pd.read_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-MTBREF_deduped_matecig_mq10_perbase_genomecov.txt', sep='\t', header=None)
mtb20_perbase_cov = pd.read_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-H37Rv20snps_deduped_matecig_mq10_perbase_genomecov.txt', sep='\t', header=None)
mtb200_perbase_cov = pd.read_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-H37Rv200snps_deduped_matecig_mq10_perbase_genomecov.txt', sep='\t', header=None)
mtb2000_perbase_cov = pd.read_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-H37Rv2000snps_deduped_matecig_mq10_perbase_genomecov.txt', sep='\t', header=None)
mtb20003_perbase_cov = pd.read_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-H37Rv20003snps_deduped_matecig_mq10_perbase_genomecov.txt', sep='\t', header=None)
mtbvar1_perbase_cov = pd.read_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-MTBVAR1_deduped_matecig_mq10_perbase_genomecov.txt', sep='\t', header=None)
mtbvar2_perbase_cov = pd.read_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-MTBVAR2_deduped_matecig_mq10_perbase_genomecov.txt', sep='\t', header=None)

In [7]:
mtbref_perbase_cov.rename(columns={0:'chr', 1:'pos', 2:'cov'}, inplace=True)
mtb20_perbase_cov.rename(columns={0:'chr', 1:'pos', 2:'cov'}, inplace=True)
mtb200_perbase_cov.rename(columns={0:'chr', 1:'pos', 2:'cov'}, inplace=True)
mtb2000_perbase_cov.rename(columns={0:'chr', 1:'pos', 2:'cov'}, inplace=True)
mtb20003_perbase_cov.rename(columns={0:'chr', 1:'pos', 2:'cov'}, inplace=True)
mtbvar1_perbase_cov.rename(columns={0:'chr', 1:'pos', 2:'cov'}, inplace=True)
mtbvar2_perbase_cov.rename(columns={0:'chr', 1:'pos', 2:'cov'}, inplace=True)

In [13]:
mtb20_perbase_cov

,chr,pos,cov
0,NC_000962.3,1,1
1,NC_000962.3,2,1
2,NC_000962.3,3,1
3,NC_000962.3,4,1
4,NC_000962.3,5,1
...,...,...,...
4411527,NC_000962.3,4411528,1
4411528,NC_000962.3,4411529,1
4411529,NC_000962.3,4411530,0
4411530,NC_000962.3,4411531,0


#### Get mutations list

In [10]:
mtbref_mut = pd.read_csv('/Users/yeemayseah/Documents/Repos/MTB-evaluation/output/variants/TEST-MTBREF.csv')
mtb20_mut = pd.read_csv('/Users/yeemayseah/Documents/Repos/MTB-evaluation/output/variants/TEST-H37Rv20snps_normalized.vcf.csv')
mtb200_mut = pd.read_csv('/Users/yeemayseah/Documents/Repos/MTB-evaluation/output/variants/TEST-H37Rv200snps_normalized.vcf.csv')
mtb2000_mut = pd.read_csv('/Users/yeemayseah/Documents/Repos/MTB-evaluation/output/variants/TEST-H37Rv2000snps_normalized.vcf.csv')
mtb20003_mut = pd.read_csv('/Users/yeemayseah/Documents/Repos/MTB-evaluation/output/variants/TEST-H37Rv20003snps_normalized.vcf.csv')
mtbvar1_mut = pd.read_csv('/Users/yeemayseah/Documents/Repos/MTB-evaluation/output/variants/TEST-MTBVAR1_normalized.vcf.csv')
mtbvar2_mut = pd.read_csv('/Users/yeemayseah/Documents/Repos/MTB-evaluation/output/variants/TEST-MTBVAR2_normalized.vcf.csv')

In [12]:
mtb20_mut

,CHROM,POS,REF,ALT,TEST-H37Rv20snps
0,NC_000962.3,221290,A,T,1
1,NC_000962.3,441140,A,T,1
2,NC_000962.3,662258,A,T,1
3,NC_000962.3,883810,T,C,1
4,NC_000962.3,1104578,T,G,1
5,NC_000962.3,1325482,C,A,1
6,NC_000962.3,1545966,T,G,1
7,NC_000962.3,1766662,C,A,1
8,NC_000962.3,1987865,G,A,1
9,NC_000962.3,2207480,C,G,1


#### Merge mutations list with coverage and MAPQ data

In [15]:
mtb20_mut_cov = pd.merge(mtb20_mut, mtb20_perbase_cov, how='inner', left_on='POS', right_on='pos')

In [17]:
mtb20_mut_cov_mapq = pd.merge(mtb20_mut_cov, mtb20, how='inner', left_on='POS', right_on='end')

In [65]:
filt_mtb20_mut_cov_mapq = mtb20_mut_cov_mapq[['CHROM', 'POS', 'REF', 'ALT', 'cov', 'min_mapq', 'max_mapq', 'median_mapq', 'count_mapq']].copy()

In [66]:
filt_mtb20_mut_cov_mapq.loc[filt_mtb20_mut_cov_mapq.min_mapq == '.', 'min_mapq'] = np.nan
filt_mtb20_mut_cov_mapq.loc[filt_mtb20_mut_cov_mapq.max_mapq == '.', 'max_mapq'] = np.nan
filt_mtb20_mut_cov_mapq.loc[filt_mtb20_mut_cov_mapq.median_mapq == '.', 'median_mapq'] = np.nan

In [96]:
filt_mtb20_mut_cov_mapq = filt_mtb20_mut_cov_mapq.astype({'min_mapq': 'float', 'max_mapq': 'float', 'median_mapq': 'float'})

In [129]:
filt_mtb20_mut_cov_mapq.to_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-H37Rv20snps_cov_mq.csv', index=False)

In [135]:
filt_mtb20_mut_cov_mapq

,CHROM,POS,REF,ALT,cov,min_mapq,max_mapq,median_mapq,count_mapq
0,NC_000962.3,221290,A,T,31,60.0,60.0,60.0,31
1,NC_000962.3,441140,A,T,42,60.0,60.0,60.0,42
2,NC_000962.3,662258,A,T,46,60.0,60.0,60.0,46
3,NC_000962.3,883810,T,C,39,60.0,60.0,60.0,39
4,NC_000962.3,1104578,T,G,35,60.0,60.0,60.0,35
5,NC_000962.3,1325482,C,A,36,60.0,60.0,60.0,36
6,NC_000962.3,1545966,T,G,48,60.0,60.0,60.0,48
7,NC_000962.3,1766662,C,A,54,60.0,60.0,60.0,54
8,NC_000962.3,1987865,G,A,4,12.0,40.0,35.0,4
9,NC_000962.3,2207480,C,G,44,60.0,60.0,60.0,44


In [98]:
mtb200_mut_cov = pd.merge(mtb200_mut, mtb200_perbase_cov, how='inner', left_on='POS', right_on='pos')
mtb200_mut_cov_mapq = pd.merge(mtb200_mut_cov, mtb200, how='inner', left_on='POS', right_on='end')
filt_mtb200_mut_cov_mapq = mtb200_mut_cov_mapq[['CHROM', 'POS', 'REF', 'ALT', 'cov', 'min_mapq', 'max_mapq', 'median_mapq', 'count_mapq']].copy()

In [99]:
filt_mtb200_mut_cov_mapq.loc[filt_mtb200_mut_cov_mapq.min_mapq == '.', 'min_mapq'] = np.nan
filt_mtb200_mut_cov_mapq.loc[filt_mtb200_mut_cov_mapq.max_mapq == '.', 'max_mapq'] = np.nan
filt_mtb200_mut_cov_mapq.loc[filt_mtb200_mut_cov_mapq.median_mapq == '.', 'median_mapq'] = np.nan

In [100]:
filt_mtb200_mut_cov_mapq = filt_mtb200_mut_cov_mapq.astype({'min_mapq': 'float', 'max_mapq': 'float', 'median_mapq': 'float'})

In [130]:
filt_mtb200_mut_cov_mapq.to_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-H37Rv200snps_cov_mq.csv', index=False)

In [103]:
mtb2000_mut_cov = pd.merge(mtb2000_mut, mtb2000_perbase_cov, how='inner', left_on='POS', right_on='pos')
mtb2000_mut_cov_mapq = pd.merge(mtb2000_mut_cov, mtb2000, how='inner', left_on='POS', right_on='end')
filt_mtb2000_mut_cov_mapq = mtb2000_mut_cov_mapq[['CHROM', 'POS', 'REF', 'ALT', 'cov', 'min_mapq', 'max_mapq', 'median_mapq', 'count_mapq']].copy()

In [105]:
filt_mtb2000_mut_cov_mapq.loc[filt_mtb2000_mut_cov_mapq.min_mapq == '.', 'min_mapq'] = np.nan
filt_mtb2000_mut_cov_mapq.loc[filt_mtb2000_mut_cov_mapq.max_mapq == '.', 'max_mapq'] = np.nan
filt_mtb2000_mut_cov_mapq.loc[filt_mtb2000_mut_cov_mapq.median_mapq == '.', 'median_mapq'] = np.nan

In [106]:
filt_mtb2000_mut_cov_mapq = filt_mtb2000_mut_cov_mapq.astype({'min_mapq': 'float', 'max_mapq': 'float', 'median_mapq': 'float'})

In [131]:
filt_mtb2000_mut_cov_mapq.to_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-H37Rv2000snps_cov_mq.csv', index=False)

In [107]:
mtb20003_mut_cov = pd.merge(mtb20003_mut, mtb20003_perbase_cov, how='inner', left_on='POS', right_on='pos')
mtb20003_mut_cov_mapq = pd.merge(mtb20003_mut_cov, mtb20003, how='inner', left_on='POS', right_on='end')
filt_mtb20003_mut_cov_mapq = mtb20003_mut_cov_mapq[['CHROM', 'POS', 'REF', 'ALT', 'cov', 'min_mapq', 'max_mapq', 'median_mapq', 'count_mapq']].copy()

In [108]:
filt_mtb20003_mut_cov_mapq.loc[filt_mtb20003_mut_cov_mapq.min_mapq == '.', 'min_mapq'] = np.nan
filt_mtb20003_mut_cov_mapq.loc[filt_mtb20003_mut_cov_mapq.max_mapq == '.', 'max_mapq'] = np.nan
filt_mtb20003_mut_cov_mapq.loc[filt_mtb20003_mut_cov_mapq.median_mapq == '.', 'median_mapq'] = np.nan

In [109]:
filt_mtb20003_mut_cov_mapq = filt_mtb20003_mut_cov_mapq.astype({'min_mapq': 'float', 'max_mapq': 'float', 'median_mapq': 'float'})

In [132]:
filt_mtb20003_mut_cov_mapq.to_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-H37Rv20003snps_cov_mq.csv', index=False)

In [128]:
filt_mtb20003_mut_cov_mapq[filt_mtb20003_mut_cov_mapq['cov'] <10]

,CHROM,POS,REF,ALT,cov,min_mapq,max_mapq,median_mapq,count_mapq
470,NC_000962.3,103959,G,T,0,NaN,NaN,NaN,0
471,NC_000962.3,104161,A,G,0,NaN,NaN,NaN,0
472,NC_000962.3,104421,G,C,0,NaN,NaN,NaN,0
473,NC_000962.3,104639,C,A,2,40.0,40.0,40.0,2
1830,NC_000962.3,400376,G,A,0,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...
19575,NC_000962.3,4318607,G,T,0,NaN,NaN,NaN,0
19576,NC_000962.3,4318841,T,A,0,NaN,NaN,NaN,0
19577,NC_000962.3,4318948,C,T,0,NaN,NaN,NaN,0
19578,NC_000962.3,4319156,A,G,0,NaN,NaN,NaN,0


In [112]:
mtbvar1_mut_cov = pd.merge(mtbvar1_mut, mtbvar1_perbase_cov, how='inner', left_on='POS', right_on='pos')
mtbvar1_mut_cov_mapq = pd.merge(mtbvar1_mut_cov, mtbvar1, how='inner', left_on='POS', right_on='end')
filt_mtbvar1_mut_cov_mapq = mtbvar1_mut_cov_mapq[['CHROM', 'POS', 'REF', 'ALT', 'cov', 'min_mapq', 'max_mapq', 'median_mapq', 'count_mapq']].copy()

In [113]:
filt_mtbvar1_mut_cov_mapq.loc[filt_mtbvar1_mut_cov_mapq.min_mapq == '.', 'min_mapq'] = np.nan
filt_mtbvar1_mut_cov_mapq.loc[filt_mtbvar1_mut_cov_mapq.max_mapq == '.', 'max_mapq'] = np.nan
filt_mtbvar1_mut_cov_mapq.loc[filt_mtbvar1_mut_cov_mapq.median_mapq == '.', 'median_mapq'] = np.nan

In [114]:
filt_mtbvar1_mut_cov_mapq = filt_mtbvar1_mut_cov_mapq.astype({'min_mapq': 'float', 'max_mapq': 'float', 'median_mapq': 'float'})

In [133]:
filt_mtbvar1_mut_cov_mapq.to_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-MTBVAR1_cov_mq.csv', index=False)

In [116]:
mtbvar2_mut_cov = pd.merge(mtbvar2_mut, mtbvar2_perbase_cov, how='inner', left_on='POS', right_on='pos')
mtbvar2_mut_cov_mapq = pd.merge(mtbvar2_mut_cov, mtbvar2, how='inner', left_on='POS', right_on='end')
filt_mtbvar2_mut_cov_mapq = mtbvar2_mut_cov_mapq[['CHROM', 'POS', 'REF', 'ALT', 'cov', 'min_mapq', 'max_mapq', 'median_mapq', 'count_mapq']].copy()

In [117]:
filt_mtbvar2_mut_cov_mapq.loc[filt_mtbvar2_mut_cov_mapq.min_mapq == '.', 'min_mapq'] = np.nan
filt_mtbvar2_mut_cov_mapq.loc[filt_mtbvar2_mut_cov_mapq.max_mapq == '.', 'max_mapq'] = np.nan
filt_mtbvar2_mut_cov_mapq.loc[filt_mtbvar2_mut_cov_mapq.median_mapq == '.', 'median_mapq'] = np.nan

In [118]:
filt_mtbvar2_mut_cov_mapq = filt_mtbvar2_mut_cov_mapq.astype({'min_mapq': 'float', 'max_mapq': 'float', 'median_mapq': 'float'})

In [134]:
filt_mtbvar2_mut_cov_mapq.to_csv('/Users/yeemayseah/Documents/Repos/read_mapping/bedtools/TEST-MTBVAR2_cov_mq.csv', index=False)

#### Remove mutations with 0-coverage from true mutations list